# Modelo 1
-----
## Modelo clasificador basado en técnicas lineales & árboles de decisión. 
Utilizaremos este modelo como base de comparación con modelos posteriores más complejos. 

Nos servirá este modelo también para detectar las variables más significativas para la detección de fraudes.

La entrada de este modelo será el dataset 'prepared' con los datos ya preparados para su consumición.